In [4]:
import requests
from time import sleep, strftime
import random
from random import randint
from bs4 import BeautifulSoup
from selenium import webdriver
import shutil
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import pandas as pd
import datetime
import funciones_nba24
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from nba_api.stats import endpoints
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.cluster import DBSCAN 
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, make_scorer,recall_score
from sklearn import tree
from sklearn.preprocessing import LabelEncoder

In [2]:
df2 = pd.read_csv("estadisticas_tradicionales.csv")
df2["GAME_ID"].unique()

array([22000490, 22000451, 22000438, ..., 22000984, 22000957, 22000568],
      dtype=int64)

In [3]:
df2.columns

Index(['Unnamed: 0', 'GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY',
       'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'MIN', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS',
       'local_visitante', 'gan_per'],
      dtype='object')

In [4]:
#El game_id está ordenado de fecha más reciente a más antigua

In [5]:
#Los enfrentamientos entre equipos sacando el más reciente, y desde 3 en adelante


In [6]:
a=["ATL", "BOS", "CLE", "NOP", "CHI", "DAL", "DEN", "GSW", "HOU", "LAC", "LAL", "MIA", "MIL", "MIN", "BKN", "NYK", "ORL", "IND",
"PHI", "PHX", "POR", "SAC", "SAS", "OKC", "TOR", "UTA", "MEM", "WAS", "DET", "CHA"]
combinaciones_games=[]
for i in itertools.combinations(a,2):
    combinaciones_games.append(i)
len(combinaciones_games)

435

## Crear data frame con las medidas de agrupación con distintos números de enfrentamientos entre combinaciones

In [88]:
df_medidas_trad = pd.DataFrame()
for k in range(len(combinaciones_games)):     
    c1 = (combinaciones_games[k][0])
    c2 = (combinaciones_games[k][1])   

   
    df_enfrentamientos_equipo = pd.DataFrame()
    
    #Guardar todos los enfrentamientos de un par de equipos
    for j in df2["GAME_ID"].unique():
        df_partido = df2[df2["GAME_ID"]==int(j)]
        if (df_partido["TEAM_ABBREVIATION"].unique()[0]==c1 or df_partido["TEAM_ABBREVIATION"].unique()[0]==c2 ) and (df_partido["TEAM_ABBREVIATION"].unique()[1]==c2 or df_partido["TEAM_ABBREVIATION"].unique()[1]==c1):
            df_enfrentamientos_equipo = pd.concat([df_enfrentamientos_equipo, df_partido])
    
    #Más de tres partidos porque necesitamos el 4to para el test
    if len(df_enfrentamientos_equipo["GAME_ID"].unique())>3:

        for i in range(3, ( len(df_enfrentamientos_equipo["GAME_ID"].unique()) ) ):
            #HAcer data frame con los game_id i últimos
            df_enfrentamientos = df_enfrentamientos_equipo[df_enfrentamientos_equipo["GAME_ID"].isin(df_enfrentamientos_equipo["GAME_ID"].unique()[-i:])]

            media = df_enfrentamientos.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).mean()    
            std = df_enfrentamientos.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).std()
            maxim = df_enfrentamientos.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).max()
            minim = df_enfrentamientos.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).min()
            suma = df_enfrentamientos.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).sum()
            med = df_enfrentamientos.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).median()
            q1 = df_enfrentamientos.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).quantile(0.25)
            q2 = df_enfrentamientos.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).quantile(0.75)
            
            #Concatenación horizontal
            concatenacion = pd.concat([media, std, maxim, minim, suma, med, q1, q2], axis=1)
            concatenacion["numero_enfrentamientos"]=i
            concatenacion["matchup"]=c1+"vs"+c2
            # Concatenación vertical
            df_medidas_trad = pd.concat([df_medidas_trad, concatenacion])

    else:
        media = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).mean()    
        std = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).std()
        maxim = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).max()
        minim = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).min()
        suma = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).sum()
        med = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).median()
        q1 = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).quantile(0.25)
        q2 = df_enfrentamientos_equipo.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).quantile(0.75)

        concatenacion = pd.concat([media, std, maxim, minim, suma, med, q1, q2], axis=1)
        concatenacion["numero_enfrentamientos"]= len(df_enfrentamientos_equipo["GAME_ID"].unique())
        concatenacion["matchup"]=c1+"vs"+c2

        df_medidas_trad = pd.concat([df_medidas_trad, concatenacion])

## Guardar en csv el df anterior

In [98]:
df_medidas_trad.drop("Unnamed: 0", axis=1, inplace=True)

In [100]:
df_medidas_trad.to_csv('estadisticas_tradicionales_agrupadas2.csv', header=True, index=True)

In [7]:
df = pd.read_csv("estadisticas_tradicionales_agrupadas2.csv")

### ELiminar columnas

In [8]:
columnas_eliminar=[]
for j in ["gan_per", "NICK", "local_visitante", "GAME_ID", "TEAM_CITY","FGM", "FG3M", "FTM", "OREB", "BLK"  ]:
    for i in df.columns:
        if i.startswith(j):
            columnas_eliminar.append(i)
    

In [9]:
df.drop(columnas_eliminar, axis=1, inplace=True)

### Cambiar el nombre de las columnas para identificar las medidas de agrupación

In [10]:
term=["_mean", "_std", "_max", "_min", "_sum", "_median", "_q25", "_q75"]
columnas_nuevas=[]
for j in term:
    b=2
    c=19
    for i in df.columns[b:c]:
        columnas_nuevas.append(i+j)
    b+=17
    c+=17

In [11]:
for i in range(len(df.columns[2:-2].values)):
    df.columns[2:-2].values[i]=columnas_nuevas[i]

### Pasar a cero las std que tienen nan

In [12]:
df=df.fillna(0)

## Asignar a cada jugador la eqtiqueta ganar perder 

In [16]:
df.to_csv('estadisticas_tradicionales_agrupadas2_limpias.csv', header=True, index=True)

In [127]:
#Encontrar todos los partidos entre una combinación
lista_total=[]
lista=[]
for k in range(len(combinaciones_games)):     
    c1 = (combinaciones_games[k][0])
    c2 = (combinaciones_games[k][1])   


    df_enfrentamientos_equipo = pd.DataFrame()
    #Encontrar todos los partidos entre una combinación
    for j in df2["GAME_ID"].unique():
        df_partido = df2[df2["GAME_ID"]==int(j)]
        if (df_partido["TEAM_ABBREVIATION"].unique()[0]==c1 or df_partido["TEAM_ABBREVIATION"].unique()[0]==c2 ) and (df_partido["TEAM_ABBREVIATION"].unique()[1]==c2 or df_partido["TEAM_ABBREVIATION"].unique()[1]==c1):
            df_enfrentamientos_equipo = pd.concat([df_enfrentamientos_equipo, df_partido])


    #Encuentra los game_id del partido próximo, que se quiere predecir, desde el 4to partido en adelante,
    #la etiqueta de ganar perder del próximo partido se le quiere poner a los enfrentamientos anteriores


    if -3 + len(df_enfrentamientos_equipo["GAME_ID"].unique()) >1:

        for i in range(3, len(df_enfrentamientos_equipo["GAME_ID"].unique()) ):
            ids = df_enfrentamientos_equipo["GAME_ID"].unique()[-i-1]

            df_game_proximo = df_enfrentamientos_equipo[df_enfrentamientos_equipo["GAME_ID"] == ids]  #4t0  

            #Todos los resúmenes de enfrentamientos anteriores de una combinación 
            df_matchup = df[df["matchup"]==c1+"vs"+c2]

            #El matchup filtrado por número de enfrentamientos
            df_matchup[df_matchup["numero_enfrentamientos"]==i]#i==3

            def g_p(x):
                #Obtenemos el nombre el equipo ganador, devolver "G"
                if x== df_game_proximo[df_game_proximo["gan_per"]=="G"]["TEAM_ABBREVIATION"].values[0]:
                    return "G"
                else:
                    return "P"
            lista = df_matchup[df_matchup["numero_enfrentamientos"]==i]["TEAM_ABBREVIATION"].apply(g_p)  

            lista_total.extend(lista)

    else:
        df_matchup = df[df["matchup"]==c1+"vs"+c2]
        len(df_matchup)
        for i in range(len(df_matchup)):
            lista_total.append("none")



In [130]:
len(lista_total)
len(df)
df["target"]=lista_total

# Guardar el df limpio agrupado y con target

In [1]:
df.to_csv('estadisticas_tradicionales_agrupadas2_limpias_target.csv', header=True, index=True)

In [82]:
df = pd.read_csv('estadisticas_tradicionales_agrupadas2_limpias_target.csv')

In [83]:
df

,Unnamed: 0,TEAM_ABBREVIATION,PLAYER_NAME,TEAM_ID_mean,PLAYER_ID_mean,MIN_mean,FGA_mean,FG_PCT_mean,FG3A_mean,FG3_PCT_mean,...,REB_q75,AST_q75,STL_q75,TO_q75,PF_q75,PTS_q75,PLUS_MINUS_q75,numero_enfrentamientos,matchup,target
0,0,ATL,Brandon Goodwin,1.610613e+09,1629164.0,4.625000,2.000000,0.333500,0.500000,0.500000,...,0.00,0.00,0.00,0.75,0.00,3.75,6.5,3,ATLvsBOS,none
1,1,ATL,Bruno Fernando,1.610613e+09,1628981.0,5.977778,0.666667,0.333333,0.000000,0.000000,...,2.50,0.00,0.00,0.50,1.00,1.00,-0.5,3,ATLvsBOS,none
2,2,ATL,Cam Reddish,1.610613e+09,1629629.0,32.450000,10.500000,0.428000,3.000000,0.250000,...,5.75,2.25,4.25,0.75,2.75,12.00,4.5,3,ATLvsBOS,none
3,3,ATL,Clint Capela,1.610613e+09,203991.0,28.394444,11.000000,0.685000,0.000000,0.000000,...,14.50,1.00,1.00,1.50,2.50,24.00,11.0,3,ATLvsBOS,none
4,4,ATL,Danilo Gallinari,1.610613e+09,201568.0,27.522222,14.333333,0.395667,8.333333,0.484000,...,4.00,2.50,1.50,3.00,2.50,23.50,17.0,3,ATLvsBOS,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14553,14553,DET,Saddiq Bey,1.610613e+09,1630180.0,35.566667,14.666667,0.288000,10.333333,0.353667,...,8.00,4.00,1.00,0.00,1.50,18.00,1.0,3,DETvsCHA,none
14554,14554,DET,Sekou Doumbouya,1.610613e+09,1629635.0,22.127778,8.333333,0.333333,2.333333,0.133333,...,6.50,1.50,0.50,1.00,2.00,14.50,-7.0,3,DETvsCHA,none
14555,14555,DET,Svi Mykhailiuk,1.610613e+09,1629004.0,26.200000,11.000000,0.364000,10.000000,0.300000,...,4.00,8.00,2.00,1.00,2.00,11.00,3.0,3,DETvsCHA,none
14556,14556,DET,Tyler Cook,1.610613e+09,1629076.0,12.150000,4.500000,0.333500,0.000000,0.000000,...,5.25,1.50,0.00,0.00,0.75,9.00,-2.5,3,DETvsCHA,none


### EL objetivo de modelo es predecir los partidos de playoff, a partir de los partidos de la fase regular. Por tanto los equipos que no pasan a play off no aportan información al modelo, serán borrados del df

In [84]:
df_modelo = df[df["target"]!="none"]

In [85]:
df_modelo

,Unnamed: 0,TEAM_ABBREVIATION,PLAYER_NAME,TEAM_ID_mean,PLAYER_ID_mean,MIN_mean,FGA_mean,FG_PCT_mean,FG3A_mean,FG3_PCT_mean,...,REB_q75,AST_q75,STL_q75,TO_q75,PF_q75,PTS_q75,PLUS_MINUS_q75,numero_enfrentamientos,matchup,target
392,392,ATL,Bogdan Bogdanovic,1.610613e+09,203992.0,30.450000,11.000000,0.377000,6.500000,0.366500,...,5.00,3.75,2.25,3.25,3.75,15.75,-13.00,3,ATLvsNYK,G
393,393,ATL,Brandon Goodwin,1.610613e+09,1629164.0,13.600000,3.500000,0.166500,2.000000,0.250000,...,0.75,0.75,0.00,0.00,0.75,5.25,-13.75,3,ATLvsNYK,G
394,394,ATL,Cam Reddish,1.610613e+09,1629629.0,33.000000,11.500000,0.348500,7.000000,0.291500,...,5.50,1.75,0.00,2.75,3.50,14.50,-4.25,3,ATLvsNYK,G
395,395,ATL,Clint Capela,1.610613e+09,203991.0,36.083333,11.666667,0.685333,0.000000,0.000000,...,20.00,1.00,1.50,1.00,3.00,20.00,0.50,3,ATLvsNYK,G
396,396,ATL,Danilo Gallinari,1.610613e+09,201568.0,26.016667,5.000000,0.600000,4.000000,0.750000,...,5.00,0.00,0.00,0.00,5.00,16.00,-14.00,3,ATLvsNYK,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14310,14310,UTA,Mike Conley,1.610613e+09,201144.0,33.758333,14.500000,0.433000,7.333333,0.456667,...,5.75,10.25,1.00,2.75,3.75,25.00,11.00,7,UTAvsMEM,G
14311,14311,UTA,Miye Oni,1.610613e+09,1629671.0,10.187500,2.250000,0.083250,2.250000,0.083250,...,1.25,1.00,1.00,0.00,2.25,0.75,5.75,7,UTAvsMEM,G
14312,14312,UTA,Royce O'Neale,1.610613e+09,1626220.0,33.159524,6.142857,0.487143,4.285714,0.453000,...,9.00,3.00,1.50,1.50,4.00,11.00,9.50,7,UTAvsMEM,G
14313,14313,UTA,Rudy Gobert,1.610613e+09,203497.0,31.980952,8.857143,0.733571,0.142857,0.000000,...,14.00,2.00,0.00,2.00,4.50,19.00,13.00,7,UTAvsMEM,G


## Dejar el último número de enfrentamientos como test, los anteriores como train

### Contrucción del x_test, e y_test con el último número de enfrentamientos

In [86]:
x_prueba =pd.DataFrame()
for k in range(len(combinaciones_games)):     
    c1 = (combinaciones_games[k][0])
    c2 = (combinaciones_games[k][1])     

    df_matchup_pof = df_modelo[df_modelo["matchup"]==c1+"vs"+c2]#["numero_enfrentamientos"]==maximo
    maximo = df_matchup_pof["numero_enfrentamientos"].max()


    x_prueba = pd.concat([x_prueba, df_matchup_pof[df_matchup_pof["numero_enfrentamientos"]==maximo]])

In [87]:
x_test = x_prueba.iloc[:,:-1]
y_test =  x_prueba.iloc[:,-1]


### Contrucción del x_train e y_train con los anteriores al último número de enfrentamientos

In [88]:
x_entrenamiento = pd.DataFrame()
for k in range(len(combinaciones_games)):     
    c1 = (combinaciones_games[k][0])
    c2 = (combinaciones_games[k][1])   


    df_matchup_pof = df_modelo[df_modelo["matchup"]==c1+"vs"+c2]
    indices = df_matchup_pof["numero_enfrentamientos"].unique()[:-1]

    x_entrenamiento = pd.concat([x_entrenamiento, df_matchup_pof[df_matchup_pof["numero_enfrentamientos"].isin(indices)] ])

In [89]:
x_train = x_entrenamiento.iloc[:,:-1]
y_train = x_entrenamiento.iloc[:,-1]


## Label encoding para matchup, eliminar team_Abreviation y player name, porque tenemos id de las 2

In [90]:
x_train.drop(["TEAM_ABBREVIATION","PLAYER_NAME" ], axis=1, inplace=True)
x_test.drop(["TEAM_ABBREVIATION","PLAYER_NAME" ], axis=1, inplace=True)

In [91]:
#label encoding
class_labelEncoding = LabelEncoder()
class_labelEncoding.fit(x_train["matchup"].values)

clase_train = class_labelEncoding.transform(x_train["matchup"].values)
clase_test = class_labelEncoding.transform(x_test["matchup"].values)



In [92]:
x_train["matchup"]=clase_train
x_test["matchup"]=clase_test

### Escalar datos

In [93]:
scaler = MinMaxScaler()
x_train_esc = scaler.fit_transform(x_train)
x_test_esc  = scaler.transform(x_train)

# Modelo random forest

## Grid Search

In [264]:
params={'n_estimators': [100,200,300],#Así se le asignan las posibilidades del valor que pueda tomat el parámetro
        'criterion':['gini','entropy'],
        #'max_depth': [3,4,5],# Maxima pofundidad del arbol
        'max_features': [2, 138], # numero de features a considerar en cada split
        'max_leaf_nodes': [16], # maximo de nodos del arbol
        #'min_impurity_decrease' : [0.02,0.3], # un nuevo nodo se hará si al hacerse se decrece la impurity en un threshold por encima del valor
        'min_samples_split': [15,30] # The minimum number of samples required to split an internal node
        }

In [265]:
# creamos la GridSeacrh
#Esta celda como pa tomar desiciones, da las buenas combinaciones y los scores

scorers = {"f1_macro","accuracy","recall_macro"}#las métricas que quiero
clf = RandomForestClassifier()
grid_solver = GridSearchCV(estimator = clf, #el clasificador
                   param_grid = params, #el diccionario params
                   scoring=scorers,# las métricas
                   cv = 5,#hace k fold
                   refit="accuracy",#métrica pa mejorar, la que más importa
                   n_jobs=-1)

model_result = grid_solver.fit(x_train,y_train)

print(model_result.cv_results_["mean_test_recall_macro"].mean())
print(model_result.cv_results_["mean_test_f1_macro"].mean())
print(model_result.cv_results_["mean_test_accuracy"].mean())
print(model_result.best_score_)
print(model_result.best_params_)

0.5316789227259882
0.5291351333168329
0.5320944906124961
0.5588104032009851
{'criterion': 'entropy', 'max_features': 138, 'max_leaf_nodes': 16, 'min_samples_split': 15, 'n_estimators': 300}


In [260]:
x_train.columns.shape

(138,)

### Grid Serach2

In [262]:
params={'n_estimators': [300,400,500],#Así se le asignan las posibilidades del valor que pueda tomat el parámetro
        'criterion':['gini','entropy'],
        #'max_depth': [3,4,5],# Maxima pofundidad del arbol
        'max_features': [2, 138], # numero de features a considerar en cada split
        'max_leaf_nodes': [16], # maximo de nodos del arbol
        #'min_impurity_decrease' : [0.02,0.3], # un nuevo nodo se hará si al hacerse se decrece la impurity en un threshold por encima del valor
        'min_samples_split': [15,30] # The minimum number of samples required to split an internal node
        }

In [263]:
# creamos la GridSeacrh
#Esta celda como pa tomar desiciones, da las buenas combinaciones y los scores

scorers = {"f1_macro","accuracy","recall_macro"}#las métricas que quiero
clf = RandomForestClassifier()
grid_solver = GridSearchCV(estimator = clf, #el clasificador
                   param_grid = params, #el diccionario params
                   scoring=scorers,# las métricas
                   cv = 5,#hace k fold
                   refit="accuracy",#métrica pa mejorar, la que más importa
                   n_jobs=-1)

model_result = grid_solver.fit(x_train,y_train)

print(model_result.cv_results_["mean_test_recall_macro"].mean())
print(model_result.cv_results_["mean_test_f1_macro"].mean())
print(model_result.cv_results_["mean_test_accuracy"].mean())
print(model_result.best_score_)
print(model_result.best_params_)

0.5323978464189233
0.5301270196066382
0.5328353595978249
0.5493782702369959
{'criterion': 'entropy', 'max_features': 138, 'max_leaf_nodes': 16, 'min_samples_split': 30, 'n_estimators': 300}


### Mejores parámetros random forest2

In [ ]:
{'criterion': 'entropy', 'max_features': 138, 'max_leaf_nodes': 16, 'min_samples_split': 30, 'n_estimators': 300}

## Modelo Random Forest

In [ ]:
{'criterion': 'entropy', 'max_features': 138, 'max_leaf_nodes': 16, 'min_samples_split': 30, 'n_estimators': 300}

In [94]:
{'criterion': 'entropy', 'max_features': 138, 'max_leaf_nodes': 16, 'min_samples_split': 30, 'n_estimators': 300}
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators= 300,
                             criterion= "entropy",
                             #max_depth= 5,
                             max_features= 138,
                             max_leaf_nodes= 16,
                             #min_impurity_decrease= 0.1,
                             min_samples_split= 30,
                             bootstrap=False,
                             random_state=123456789,
                             n_jobs= -1)

clf.fit(x_train, y_train)
yhat = clf.predict(x_test)
print("Accuracy: ", accuracy_score(y_test,yhat))


Accuracy:  0.5035971223021583


In [269]:
x_prueba["predicciones"]=yhat

In [540]:
x_prueba.head(2)

,TEAM_ABBREVIATION,PLAYER_NAME,TEAM_ID_mean,PLAYER_ID_mean,MIN_mean,FGA_mean,FG_PCT_mean,FG3A_mean,FG3_PCT_mean,FTA_mean,...,STL_q75,TO_q75,PF_q75,PTS_q75,PLUS_MINUS_q75,numero_enfrentamientos,matchup,target,predicciones,predicciones_proba
507,ATL,Bogdan Bogdanovic,1.610613e+09,203992.0,33.991667,13.5,0.4110,8.0,0.420833,0.0,...,2.0,1.75,4.00,18.00,15.50,7,ATLvsNYK,G,P,0.429789
508,ATL,Brandon Goodwin,1.610613e+09,1629164.0,13.600000,3.5,0.1665,2.0,0.250000,1.0,...,0.0,0.00,0.75,5.25,-13.75,7,ATLvsNYK,G,G,0.537644


### Generar listas por game_id de los equipos ganadores reales y predichos

In [335]:
valores_predichos=[]
valores_reales=[]
for k in x_prueba["matchup"]:     
    c1 = k.split("vs")[0]
    c2 = k.split("vs")[1]

    try:
        g_c1 = x_prueba[x_prueba["matchup"]==c1+"vs"+c2][["TEAM_ABBREVIATION","predicciones"]].value_counts()[c1]["G"]
    except:
        g_c1=0

    try:
        p_c1 = x_prueba[x_prueba["matchup"]==c1+"vs"+c2][["TEAM_ABBREVIATION","predicciones"]].value_counts()[c1]["P"]
    except:
        p_c1=0

    try:
        g_c2 = x_prueba[x_prueba["matchup"]==c1+"vs"+c2][["TEAM_ABBREVIATION","predicciones"]].value_counts()[c2]["G"]
    except:
        g_c2 = 0

    try:
        p_c2 = x_prueba[x_prueba["matchup"]==c1+"vs"+c2][["TEAM_ABBREVIATION","predicciones"]].value_counts()[c2]["P"]
    except:
        p_c2 = 0

    if (g_c1 > g_c2) and (p_c1 < p_c2):
        valores_predichos.append(c1)
    elif (g_c1 > g_c2) and (p_c1 == p_c2):
        valores_predichos.append(c1)
    elif (g_c1 == g_c2) and (p_c1 < p_c2):
        valores_predichos.append(c1)
    elif (g_c1 == g_c2) and (p_c1 == p_c2):
        valores_predichos.append("Empate")
    else:
        valores_predichos.append(c2)

    #Ganadores Reales
    df_m = x_prueba[x_prueba["matchup"]==c1+"vs"+c2]
    valores_reales.append(df_m[df_m["target"]=="G"]["TEAM_ABBREVIATION"].values[0])

In [ ]:
yhat = clf.predict(x_test)

In [338]:
resultados = pd.DataFrame({"Reales":valores_reales, "Predichos":valores_predichos})

In [345]:
comparacion = np.where(resultados['Reales'] == resultados['Predichos'], 1, 0)
resultados["comparacion"]=comparacion
resultados

,Reales,Predichos,comparacion
0,ATL,ATL,1
1,ATL,ATL,1
2,ATL,ATL,1
3,ATL,ATL,1
4,ATL,ATL,1
...,...,...,...
412,UTA,UTA,1
413,UTA,UTA,1
414,UTA,UTA,1
415,UTA,UTA,1


In [348]:
resultados["comparacion"].value_counts(normalize=True)

1    0.834532
0    0.165468
Name: comparacion, dtype: float64

## Comparar resultados con probabilidad

In [483]:
yhat_proba = clf.predict_proba(x_test)

In [492]:
prob_ganar_pred = [yhat_proba[i][0] for i in range(len(yhat_proba))]


In [493]:
x_prueba["predicciones_proba"]=prob_ganar_pred

In [515]:
#Una combinación
k="ATLvsPHI"
c1 = k.split("vs")[0]
c2 = k.split("vs")[1]

mat = x_prueba[x_prueba["matchup"]==k]#["TEAM_ABBREVIATION"]=="ATL"
prob_c1 = mat[mat["TEAM_ABBREVIATION"]==c1].groupby("TEAM_ABBREVIATION").sum()["predicciones_proba"]/len(mat[mat["TEAM_ABBREVIATION"]==c1])


mat = x_prueba[x_prueba["matchup"]==k]#["TEAM_ABBREVIATION"]=="ATL"
prob_c2 = mat[mat["TEAM_ABBREVIATION"]==c2].groupby("TEAM_ABBREVIATION").sum()["predicciones_proba"]/len(mat[mat["TEAM_ABBREVIATION"]==c2])

if prob_c1.values[0] >  prob_c2.values[0]:
    print(c1)
else:
    print(c2)

In [512]:
mat[["TEAM_ABBREVIATION", "predicciones"]].value_counts()

TEAM_ABBREVIATION  predicciones
PHI                G               16
ATL                P                9
                   G                8
PHI                P                1
dtype: int64

In [530]:
#Todas las combinaciones
valores_predichos_proba=[]

for k in x_prueba["matchup"]:     
    
    c1 = k.split("vs")[0]
    c2 = k.split("vs")[1]


    mat = x_prueba[x_prueba["matchup"]==k]
    prob_c1 = mat[mat["TEAM_ABBREVIATION"]==c1].groupby("TEAM_ABBREVIATION").sum()["predicciones_proba"]/len(mat[mat["TEAM_ABBREVIATION"]==c1])


    mat = x_prueba[x_prueba["matchup"]==k]#["TEAM_ABBREVIATION"]=="ATL"
    prob_c2 = mat[mat["TEAM_ABBREVIATION"]==c2].groupby("TEAM_ABBREVIATION").sum()["predicciones_proba"]/len(mat[mat["TEAM_ABBREVIATION"]==c2])

    if prob_c1.values[0] >  prob_c2.values[0]:
        valores_predichos_proba.append(c1)
    else:
         valores_predichos_proba.append(c2)


    

In [531]:
len(valores_predichos_proba)

417

In [532]:
resultados["predichos_proba"]=valores_predichos_proba

In [535]:
comparacion_proba = np.where(resultados['Reales'] == resultados['predichos_proba'], 1, 0)

In [536]:
resultados["comparacion_proba"]=comparacion_proba

In [538]:
resultados["comparacion_proba"].value_counts()

1    241
0    176
Name: comparacion_proba, dtype: int64

In [539]:
241/(176+241)

0.5779376498800959

## Predicción de un próximo partido 

## Reentrenar con todos los datos, df_modelo

In [36]:
xtrain_entero = pd.concat([x_train, x_test])
ytrain_entero = pd.concat([y_train, y_test])

In [44]:
#Cargar
xtrain_entero.drop("Unnamed: 0", axis=1, inplace=True)

In [45]:
len(xtrain_entero.columns)

138

In [ ]:
{'criterion': 'entropy', 'max_features': 138, 'max_leaf_nodes': 16, 'min_samples_split': 30, 'n_estimators': 300}

In [46]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators= 300,
                             criterion= "entropy",
                             #max_depth= 5,
                             max_features= 138,
                             max_leaf_nodes= 16,
                             #min_impurity_decrease= 0.1,
                             min_samples_split= 30,
                             bootstrap=False,
                             random_state=123456789,
                             n_jobs= -1)

clf.fit(xtrain_entero, ytrain_entero)
#yhat = clf.predict(x_test)
#print("Accuracy: ", accuracy_score(y_test,yhat))


RandomForestClassifier(bootstrap=False, criterion='entropy', max_features=138,
                       max_leaf_nodes=16, min_samples_split=30,
                       n_estimators=300, n_jobs=-1, random_state=123456789)

## Guardar el modelo

In [47]:
import pickle
#Guardar
pickle.dump(clf, open('finalized_model.sav', 'wb'))
#Cargar
loaded_model = pickle.load(open("finalized_model.sav", 'rb'))


In [ ]:
loaded_model.predict

# Predicción de un resultado dados dos ciudades que se enfrentan:
    Recibe, las siglas de las ciudades que se enfrentan, la fecha de inicio de temporada.


In [2]:
#city_a = "LAC"
#city_b = "PHX"

def apuesta(city_a, city_b, inicio_temporada):
    inicio_temporada = "2020-12-22"
    trad = pd.DataFrame()
    df_medidas_trad = pd.DataFrame()

    nba_teams = teams.get_teams()
    # Select the dictionary for the Celtics, which contains their team ID
    celtics = [team for team in nba_teams if team['abbreviation'] == city_a][0]
    celtics_id = celtics['id']

    # Query for games where the Celtics were playing
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
    # The first DataFrame of those returned is what we want.
    games = gamefinder.get_data_frames()[0]

    games["GAME_DATE"]= games ["GAME_DATE"].astype("datetime64")#Convertimos en formato fecha el game_date
    games_temporada = games[games["GAME_DATE"]>=inicio_temporada]#Escoger los partidos dentro de la temporada
    #Filtrar por la ciudad rival
    games_temporada = games_temporada[games_temporada["MATCHUP"].isin([city_a+" @ "+city_b, city_a+" vs. "+city_b])].sort_values("GAME_DATE",ascending=False)

    #reiniciar index
    games_temporada = games_temporada.reset_index()
    games_temporada.drop("index", axis=1, inplace=True)

    games_temporada["GAME_DATE"] = games_temporada["GAME_DATE"].map(lambda x: x.strftime("%Y-%m-%d"))#Cambiamos fecha a str
    fechas_partidos = games_temporada["GAME_DATE"].values

    #Guardo en id_games los id de los juegos de la temporada entre city_a y city_b
    id_games = games_temporada["GAME_ID"].values



    #Concatenar los df de estadísticas tradicionales de cada juego
    for i in id_games:    
        # Here we access the leagueleaders module through endpoints & assign the class to "data"
        #id="0042000207"
        data = endpoints.boxscoretraditionalv2.BoxScoreTraditionalV2(i)

        # Our "data" variable now has built in functions such as creating a dataframe for our data
        df = data.player_stats.get_data_frame()

        trad = pd.concat([trad, df])

    ### Limpiar df
    #Eliminar COMMENT
    trad.drop("COMMENT", axis=1, inplace=True)

    #Eliminar START_POSITION       
    trad.drop("START_POSITION", axis=1, inplace=True)

    # Eliminar las filas con nan, son jugadores que no jugaron
    trad.dropna(axis=0, inplace = True) 

    def minutos(x):
        tiempo=[]
        for i in range(2):
            tiempo.append(int(x.split(":")[i]))
        return (tiempo[1]/60)+tiempo[0]

    trad["MIN"] = trad["MIN"].apply(minutos)

    #Agrupar medidas
    media = trad.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).mean()    
    std = trad.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).std()
    maxim = trad.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).max()
    minim = trad.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).min()
    suma = trad.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).sum()
    med = trad.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).median()
    q1 = trad.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).quantile(0.25)
    q2 = trad.groupby(["TEAM_ABBREVIATION","PLAYER_NAME"]).quantile(0.75)

    concatenacion = pd.concat([media, std, maxim, minim, suma, med, q1, q2], axis=1)
    concatenacion["numero_enfrentamientos"]= len(trad["GAME_ID"].unique())
    concatenacion["matchup"]=city_a+"vs"+city_b

    df_medidas_trad = pd.concat([df_medidas_trad, concatenacion])
    df_medidas_trad=df_medidas_trad.fillna(0)


    #Eliminar columas
    columnas_eliminar=[]
    for j in ["gan_per", "NICK", "local_visitante", "GAME_ID", "TEAM_CITY","FGM", "FG3M", "FTM", "OREB", "BLK"  ]:
        for i in df_medidas_trad.columns:
            if i.startswith(j):
                columnas_eliminar.append(i)
    df_medidas_trad.drop(columnas_eliminar, axis=1, inplace=True)  

    #Dar matchup
    df_medidas_trad['matchup'] = 18

    #Reiniciar índice
    df_medidas_trad = df_medidas_trad.reset_index()
    df_medidas_trad2 = df_medidas_trad.copy()
    df_medidas_trad2.drop(["TEAM_ABBREVIATION", "PLAYER_NAME"], axis=1, inplace=True)

    #Aplicar modelo
    #Cargar
    loaded_model = pickle.load(open("finalized_model.sav", 'rb'))
    results = loaded_model.predict(df_medidas_trad2)
   

    #Agregar resultados al df
    df_medidas_trad["resultados"]=results
    df_medidas_trad[["TEAM_ABBREVIATION", "resultados"]].value_counts()[city_a]["G"]

    #Decidir ganador

    try:
        g_c1 = df_medidas_trad[["TEAM_ABBREVIATION", "resultados"]].value_counts()[city_a]["G"]
    except:
        g_c1=0

    try:
        p_c1 = df_medidas_trad[["TEAM_ABBREVIATION", "resultados"]].value_counts()[city_a]["P"]
    except:
        p_c1=0

    try:
        g_c2 = df_medidas_trad[["TEAM_ABBREVIATION", "resultados"]].value_counts()[city_b]["G"]
    except:
        g_c2 = 0

    try:
        p_c2 = df_medidas_trad[["TEAM_ABBREVIATION", "resultados"]].value_counts()[city_b]["P"]
    except:
        p_c2 = 0

    if (g_c1 > g_c2) and (p_c1 < p_c2):
        return(city_a)
    elif (g_c1 > g_c2) and (p_c1 == p_c2):
        return(city_a)
    elif (g_c1 == g_c2) and (p_c1 < p_c2):
        return(city_a)
    elif (g_c1 == g_c2) and (p_c1 == p_c2):
        return("Empate")
    else:
        return(city_b)

In [806]:
apuesta(city_a, city_b)

'MIL'

In [5]:
apuesta("MIL", "ATL", "2020-12-22")

'MIL'

## Variables más importantes para el modelo dado un enfrentamiento

In [ ]:
from sklearn.ensemble import RandomForestClassifier
X = x_train
#clf = RandomForestClassifier(n_estimators=5000)
#clf.fit(X, y)
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

1. 'PLUS_MINUS_q75’ (10,9069%) 
2. 'numero_enfrentamientos’ (7,6754%)
3. 'FGA_std’ (6,4569%) 
4. ‘PLAYER_ID’ (6,1610%) 
5. 'AST_q25’ (5,6829%) 
6. "DREB_max“ (5,5977%)
